In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time
import random
import csv

In [2]:
ua = UserAgent()

In [3]:
# ua.random

In [4]:
headers = {
    'user-agent': ua.random}

In [5]:
BASE_URL = 'https://ru.turbo.az/autos'

In [6]:
def get_html(url):
    r = requests.get(url, headers=headers)
    return r.text

In [7]:
def get_page_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find(class_="pagination")
    string = pagination.find_all('a')[-2].get('href')
    num_of_pages = int(''.join(s for s in string if s.isdigit()))
    return num_of_pages

In [8]:
def parse(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    cars_list = soup.find_all('div', {'class': 'products-i'})
    
    link_of_cars = []
    
    for link in cars_list:
        car_link = link.a.get('href')
        if len(str(car_link)) <= 100:
            link_of_cars.append('https://ru.turbo.az' + str(car_link))
    
    labels_taglist = ['ad_region', 'ad_make_id', 'ad_model', 'ad_reg_year', 'ad_category', 'ad_color', 'ad_engine_volume', 'ad_power', 'ad_fuel_type', 'ad_mileage', 'ad_transmission', 'ad_gear', 'ad_new', 'ad_price']
    characteristics = []
    
    for link in link_of_cars:
        r2 = requests.get(link, headers=headers)
        soup2 = BeautifulSoup(r2.text, 'html.parser')
        properties_list = soup2.find_all('div', {'class': 'product-properties-value'})
        labels = soup2.find_all('label')
        labels_list = soup2.find_all('label', {'for': labels_taglist })
        update_time = soup2.find('div', {'class': 'product-statistics'})
#         print(link)
#         print(soup2.find(class_="product-text").text)
        characteristics.append({
        'Время': update_time.find_all('p')[1].text[11:],
        'Город': properties_list[0].text,
        'Марка':properties_list[1].text,
        'Модель':properties_list[2].text,
        'Год выпуска':properties_list[3].text,
        'Тип кузова':properties_list[4].text,
        'Цвет':properties_list[5].text,
        'Объем двигателя':properties_list[6].text,
        'Мощность':properties_list[7].text,
        'Тип топлива':properties_list[8].text,
        'Пробег':properties_list[9].text,
        'Коробка передач':properties_list[10].text,
        'Привод':properties_list[11].text,
        'Новая':properties_list[12].text,
        'Описание':' '.join([str(elem) for elem in soup2.find(class_="product-text").text.split('\n')]) if soup2.find(class_="product-text") is not None else 'No description',
        'Цена':properties_list[13].text
        })
        time.sleep(random.randint(1, 3))
    
    return characteristics

In [9]:
# get_page_count(get_html('https://ru.turbo.az/autos'))
# get_html(BASE_URL)
# page_count = get_page_count(get_html(BASE_URL)
# page_count
# listik = parse(get_html('https://ru.turbo.az/autos?page=1'))
# htmlka = get_html('https://ru.turbo.az/autos/4027711-lada-vaz-2107')
# soup = BeautifulSoup(htmlka, 'html.parser')
# string = ' '.join([str(elem) for elem in soup.find(class_="product-text").text.split('\n')]) if soup.find(class_="product-text") is not None else 'got'
# string
# print(soup.find(class_="product-text"))

In [10]:
i = 1
flag = True
for page in range(1,61):
    characteristics_list = []
    print('Parsing {} %'.format(int(page/60 * 100)))
    characteristics_list.extend(parse(get_html(BASE_URL + '?page={}'.format(page))))
    with open('characteristics_cars.csv', 'a', encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        if flag:
            writer.writerow(('Время публикации', 'Автор объявления', 'Название товара', 'Город', 'Марка', 'Новый', 'Доставка', 'Память', 'Описание', 'Цена', 'Контакты для связи', 'Ссылка на объявление'))
            flag = False
        for characteristic in characteristics_list:
            writer.writerow((characteristic['Время публикации'], characteristic['Автор объявления'], characteristic['Название товара'], characteristic['Город'], characteristic['Марка'], characteristic['Новый'] if 'Новый' in characteristic.keys() else '', characteristic['Доставка'] if 'Доставка' in characteristic.keys() else '', characteristic['Память'], characteristic['Описание'], characteristic['Цена'], characteristic['Контакты для связи'], characteristic['Ссылка на объявление']))
        
    

In [ ]:
page_count = get_page_count(get_html(BASE_URL))
print('Number of pages is {}'.format(page_count))
# characteristics_list = []
flag = True
while True:
    for page in range(1, 4):
        characteristics_list = []
        print('Parsing {} %'.format(int(page/3 * 100)))
        characteristics_list.extend(parse(get_html(BASE_URL + '?page={}'.format(page))))
#         time.sleep(5)
        if page%3 == 0:
            with open('characteristics.csv', 'a', encoding="utf-8") as csvfile:
                writer = csv.writer(csvfile)
                if flag:
                    writer.writerow(('Время', 'Город', 'Марка', 'Модель', 'Год выпуска', 'Тип кузова', 'Цвет', 'Объем двигателя', 'Мощность', 'Тип топлива', 'Пробег', 'Коробка передач', 'Привод', 'Новая', 'Описание', 'Цена'))
                    flag = False
                for characteristic in characteristics_list:
                    writer.writerow((characteristic['Время'], characteristic['Город'], characteristic['Марка'], characteristic['Модель'], characteristic['Год выпуска'], characteristic['Тип кузова'], characteristic['Цвет'], characteristic['Объем двигателя'], characteristic['Мощность'], characteristic['Тип топлива'], characteristic['Пробег'], characteristic['Коробка передач'], characteristic['Привод'], characteristic['Новая'], characteristic['Описание'], characteristic['Цена']))
    print('Sleeping for 10 minutes')
    time.sleep(600)
            #         print('Page {} finished'.format(page))

# for characteristics in characteristics_list:
#     characteristics

Number of pages is 1116
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes
Parsing 33 %
Parsing 66 %
Parsing 100 %
Sleeping for 10 minutes


In [ ]:
# characteristics_list

In [ ]:
# import csv
# def save(characteristics, path):
#     with open(path, 'w', encoding="utf-8") as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerow(('Город', 'Марка', 'Модель', 'Год выпуска', 'Тип кузова', 'Цвет', 'Объем двигателя', 'Мощность', 'Тип топлива', 'Пробег', 'Коробка передач', 'Привод', 'Новая', 'Описание', 'Цена'))
        
#         for characteristic in characteristics:
#             writer.writerow((characteristic['Город'], characteristic['Марка'], characteristic['Модель'], characteristic['Год выпуска'], characteristic['Тип кузова'], characteristic['Цвет'], characteristic['Объем двигателя'], characteristic['Мощность'], characteristic['Тип топлива'], characteristic['Пробег'], characteristic['Коробка передач'], characteristic['Привод'], characteristic['Новая'], characteristic['Описание'], characteristic['Цена']))

In [ ]:
# save(characteristics_list, 'characteristics.csv')